In [1]:
import torch
import torchvision.datasets as datasets # for Mist
import torchvision.transforms as transforms # Transformations we can perform on our dataset for augmentation
from torch import optim # For optimizers like SGD, Adam, etc.
from torch import nn # To inherit our neural network
from torch.utils.data import DataLoader # For management of the dataset (batches)
from tqdm import tqdm # For nice progress bar!
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import flappy_bird_gymnasium as flappy_bird
import gymnasium as gym
import random
from collections import deque, namedtuple
from preprocessing.preprocessing import preprocess_frame
from PIL import Image
from gymnasium.wrappers import RecordVideo
from static_variables import CHECKPOINTS_DIR, VIDEOS_DIR, PHOTOS_DIR, make_dirs
import os
import hashlib
from preprocessing.preprocessing import preprocess_frame
from model import DQN_CNN
from save_model.utils import save_model, load_model, record_trained_agent_video
from replay_buffer.ReplayBuffer import ReplayBuffer
from model import save_input_frames
from save_model.utils import transition

/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
make_dirs()

In [3]:
if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device = torch.device(device)
print("Device: ", device)

Device:  mps


In [4]:
# Environment
frame_skip = 4
train_every = 4   

# Training
data_type = torch.float32
batch_size = 64
T_iterations = 100000

# Replay buffer
replay_buffer_size = 30000

# DQN
alpha = 1e-4
gamma = 0.99

# Target network (soft update)
target_network_incorporation_rate = 0.005

# Epsilon-greedy
e_start = 1.0
e_end = 0.0001
k_epsilon = -1 / T_iterations * np.log(e_end / e_start)



# Early stopping parameters
avg_window = 100          # moving average window
patience = 500            # episodes to wait without improvement
min_delta = 0.5           # minimum improvement to count as progress

reward_window = deque(maxlen=avg_window)
best_avg_reward = -float("inf")
no_improve_counter = 0


In [ ]:

env = gym.make("FlappyBird-v0", render_mode="rgb_array")

policy_net = DQN_CNN(frame_skip).to(device)
target_net = DQN_CNN(frame_skip).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=alpha)
memory = ReplayBuffer(replay_buffer_size)

global_step = 0
transition_steps = 0
frame_id = 0
best_reward = -float("inf")

criterion = nn.SmoothL1Loss()

for episode in range(T_iterations):
    env.reset()
    done = False
    episode_reward = 0

    current_state, _, _ = transition(0, env, frame_skip)

    while not done:
        global_step += 1

        epsilon = e_end + (e_start - e_end) * np.exp(-k_epsilon * global_step)

        state_tensor = torch.tensor(
            current_state, dtype=torch.float32, device=device
        ).unsqueeze(0)

        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                action = policy_net(state_tensor).argmax(dim=1).item()

        next_state, reward, done = transition(action, env, frame_skip)

        memory.push(current_state, action, reward, next_state, done)
        episode_reward += reward
        current_state = next_state
        transition_steps += 1

        if len(memory) >= batch_size and global_step % train_every == 0:
            states, actions, rewards, next_states, dones = memory.sample(batch_size)

            states = torch.tensor(states, dtype=torch.float32, device=device)
            actions = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
            rewards = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
            next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
            dones = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

            q_values = policy_net(states).gather(1, actions)

            with torch.no_grad():
                next_q = target_net(next_states).max(1, keepdim=True)[0]
                target_q = rewards + gamma * next_q * (1 - dones)

            loss = criterion(q_values, target_q)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 10.0)
            optimizer.step()

            with torch.no_grad():
                for t_param, p_param in zip(
                    target_net.parameters(), policy_net.parameters()
                ):
                    t_param.data.mul_(1.0 - target_network_incorporation_rate)
                    t_param.data.add_(target_network_incorporation_rate * p_param.data)

    # ---------- episode end ----------
    reward_window.append(episode_reward)

    if len(reward_window) == avg_window:
        avg_reward = np.mean(reward_window)

        if avg_reward > best_avg_reward + min_delta:
            best_avg_reward = avg_reward
            no_improve_counter = 0

            # Save the best model
            torch.save(policy_net.state_dict(), "best_dqn.pt")
        else:
            no_improve_counter += 1

        if no_improve_counter >= patience and epsilon < 0.05:
            print(
                f"\n🛑 Early stopping!\n"
                f"Best Avg Reward: {best_avg_reward:.2f}\n"
                f"Episode: {episode}"
                f"Steps: {transition_steps}"
            )
            break

    best_reward = max(best_reward, episode_reward)

    print(
        f"Episode {episode:5d} | "
        f"Reward: {episode_reward:7.2f} | "
        f"Avg({avg_window}): {np.mean(reward_window):7.2f} | "
        f"Epsilon: {epsilon:.4f} | "
        f"Best Avg Reward: {best_avg_reward:7.2f} | "
        f"Best Reward: {best_reward:7.2f} | "
        f"Steps: {transition_steps:7d}"
    )

env.close()

/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Episode     0 | Reward:   -1.30 | Avg(100):   -1.30 | Epsilon: 0.9989 | Best Avg Reward:    -inf | Best Reward:   -1.30 | Steps:      12
Episode     1 | Reward:   -0.30 | Avg(100):   -0.80 | Epsilon: 0.9979 | Best Avg Reward:    -inf | Best Reward:   -0.30 | Steps:      23
Episode     2 | Reward:   -3.70 | Avg(100):   -1.77 | Epsilon: 0.9968 | Best Avg Reward:    -inf | Best Reward:   -0.30 | Steps:      35
Episode     3 | Reward:   -3.70 | Avg(100):   -2.25 | Epsilon: 0.9957 | Best Avg Reward:    -inf | Best Reward:   -0.30 | Steps:      47
Episode     4 | Reward:   -1.30 | Avg(100):   -2.06 | Epsilon: 0.9946 | Best Avg Reward:    -inf | Best Reward:   -0.30 | Steps:      59
Episode     5 | Reward:    1.70 | Avg(100):   -1.43 | Epsilon: 0.9935 | Best Avg Reward:    -inf | Best Reward:    1.70 | Steps:      71
Episode     6 | Reward:   -0.70 | Avg(100):   -1.33 | Epsilon: 0.9924 | Best Avg Reward:    -inf | Best Reward:    1.70 | Steps:      83
Episode     7 | Reward:   -1.30 | Avg(100

In [ ]:
path = save_model(
    policy_net,
    target_net,
    optimizer,
    dir=CHECKPOINTS_DIR,
    global_step=global_step,
    best_reward=2,
)


✅ Model saved to ../checkpoints/c1a6547f62ac26036be5c3c2eb6cd984


In [ ]:
record_trained_agent_video(
    model_path=path,
    video_dir=VIDEOS_DIR,
    frame_skip=frame_skip,
    device=device,
)


/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/stefaniancu/Documents/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation spac

🎥 Video saved in '../videos/' | Reward: 46.10
